# Revised case normalization for Hirslanden Linde 2017

Notes:

- According to this notebook: missing BfS data
- According to older notebook: 4 of 18 revised cases could be matched with the DB. 



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group, format_for_grouper_one_case, format_for_grouper
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
                     'Hirslanden Klinik Linde', '2017', 'Änderungen_LI_2017')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Klinik Linde', year='2017', sheet='Änderungen_LI_2017')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 09:01:03.975 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 18 cases for Hirslanden Klinik Linde 2017
2022-11-21 09:01:03.979 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
datum                     string
tranche                   string
case_id                   object
patient_id                object
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    string
cw-änderung möglich       str

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,657093,D594EE66C57B286C,W,72,11,657093,K430,M6208,M200,3,G08B,[],[],[],[]
1,669177,8125714B0579B1A0,M,63,8,669177,J181,J181,M100,2,E77G,"[R650, J9610]",[],[],[]
2,670466,6A0E65CA7E381F17,W,76,24,670466,C187,C187,M200,3,G02C,[R650],[],[],[]
3,674893,E120201DBEFFB939,M,73,6,674893,M4802,M4802,M200,2,I09E,[T810],[],[],[]
4,680781,F5130C1F2D9962C3,M,72,4,680781,N328,N328,M200,3,L18B,[G2090],[],[],[]
5,681347,245EC023C64BCAE9,W,73,11,681347,M511,M511,M200,3,I10C,[I422],[],[],[]
6,663181,89435BFF1A8C7B75,W,81,12,663181,M161,M161,M200,3,I46C,"[E1140, G632]",[E1190],[],[]
7,677195,D0CE1E822E9B314C,M,88,12,677195,I5013,I5013,M100,3,F62C,[J9600],[],[],[]
8,696484,2201A24926281FA3,M,72,5,696484,D410,D410,M100,3,L62C,[F339],[],[],[]
9,664975,4151972F3AAB5BC7,M,60,11,664975,I5001,I5001,M100,3,F62C,[I691],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-21 09:01:05.117 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 6177 rows from the DB, for the hospital 'Hirslanden Klinik Linde' in 2017


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,211191,681347,M511,"[G551, T813, Y849, T818, G9780, ...]",805110::20170503,"[030934::20170503, 030941::20170510, 035913::20170510]",W,73,0,0,11,0,01,2017-05-03,0,00,2017-05-14
1,211441,670466,C187,"[K9183, Z488, I1000, T813, K650, ...]",457512::20170220,"[461211::20170225, 9915::20170228, 009A14::20170220, 009A12::20170220, 459812::20170225]",W,76,0,0,24,0,01,2017-02-19,0,00,2017-03-15
2,212423,664975,I5001,"[I481, I1000, E789, E1161, F03, ...]",,[],M,60,0,0,11,0,01,2016-12-26,0,00,2017-01-06
3,213182,685780,N390,"[R650, A418, N1781, M706, M353, ...]",,[],W,75,0,0,7,0,01,2017-05-30,0,00,2017-06-06
4,213507,697356,J181,"[J981, I5012, E1191, E789, N390, ...]",,[],W,84,0,0,9,0,01,2017-08-31,0,00,2017-09-09


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 09:01:05.506 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 18 cases ...
2022-11-21 09:01:06.533 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 18 cases into: 18 revisions, 161 diagnoses rows, 41 procedure rows


In [7]:
procedures_update

,aimedic_id,code,side,date,is_grouper_relevant,is_primary
0,211191,805110,,2017-05-03,True,False
1,211191,030934,,2017-05-03,False,False
2,211191,030941,,2017-05-10,False,False
3,211191,035913,,2017-05-10,False,False
4,211441,457512,,2017-02-20,True,False
5,211441,461211,,2017-02-25,False,False
6,211441,9915,,2017-02-28,False,False
7,211441,009A14,,2017-02-20,False,False
8,211441,009A12,,2017-02-20,False,False
9,211441,459812,,2017-02-25,False,False


In [8]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,211191,I10A,2.67,2.67,4,2022-12-31
1,211441,G02A,4.10,4.10,4,2022-12-31
2,212423,F62B,1.38,1.38,4,2022-12-31
3,213182,L63B,1.04,1.04,4,2022-12-31
4,213507,E77D,1.18,1.18,4,2022-12-31
5,213570,L06A,1.96,1.96,4,2022-12-31
6,214563,E77D,1.18,1.18,4,2022-12-31
7,214809,J62A,1.41,1.41,4,2022-12-31
8,214853,I46A,2.66,2.66,4,2022-12-31
9,215064,L63D,0.67,0.67,3,2022-12-31


In [9]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 09:01:06.563 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 18 cases into the 'Revisions' table ...
2022-11-21 09:01:07.706 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 18 cases into the 'Revisions' table
2022-11-21 09:01:07.715 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 161 rows into the 'Diagnoses' table ...
2022-11-21 09:01:09.899 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 161 rows into the 'Diagnoses' table
2022-11-21 09:01:09.903 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 41 rows into the 'Procedures' table ...
2022-11-21 09:01:11.894 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 41 rows into the 'Procedures' table
